# Consignes pour le TP spaCY

## Objectifs

L'objectif du TP est d'utiliser les fonctionnalités de spaCy pour extraire de l'information pertinente d'un texte en français.
Pour ce faire, vous pouvez vous aider des notebooks utilisés lors de la séance ainsi que de la [documentation](https://spacy.io/usage/linguistic-features) en ligne.

## Prise en main de l'outil

Commençons par importer la librairie spaCy ainsi que le module Matcher : 

In [1]:
import spacy
from spacy.matcher import Matcher

Partons ensuite d'un texte d'exemple en français tiré de [Wikipédia](https://fr.wikipedia.org/wiki/ChatGPT) :

In [2]:
text = """ChatGPT (Chat Generative Pre-trained Transformer) est un prototype d'agent conversationnel utilisant l'intelligence artificielle, développé par OpenAI et spécialisé dans le dialogue.
L'agent conversationnel de ChatGPT repose sur le modèle de langage GPT d'OpenAI, et est affiné en continu grâce à l'utilisation de techniques d'apprentissage supervisé et d'apprentissage par renforcement.
ChatGPT est capable de générer des réponses à des questions, de compléter des phrases, de traduire des textes, d'écrire des articles et de tenir des conversations avec des humains. Il peut également synthétiser des textes suivant un ensemble de contraintes, telles que le ton, le style et le sujet. Il a été utilisé dans diverses applications, telles que la génération de sous-titres pour des vidéos et la création de chatbots.
En raison de ses capacités multiples, le prototype suscite également des inquiétudes. Sont craints des détournements à des fins malveillantes, des risques de plagiat dans le monde académique et de possibles suppressions d'emplois dans certains secteurs. ChatGPT soulève également des préoccupations en matière de sécurité et de confidentialité, car le modèle peut être utilisé pour générer des faux textes et des informations trompeuses.
Lancé en novembre 2022 dans une version gratuite et non connectée à Internet, ChatGPT bénéficie d'une large exposition médiatique et reçoit un accueil globalement positif, bien que son exactitude factuelle soit critiquée.
En janvier 2023, ChatGPT compte plus de 100 millions de comptes enregistrés."""

Téléchargeons le modèle spaCy pour le français (fr), de petite taille (sm = small) :

In [3]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 14.0 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


Ce modèle peut à présent être chargé en mémoire, et le texte converti en objet spaCy : 

In [4]:
nlp = spacy.load("fr_core_news_sm")
doc = nlp(text)

Définissons un pattern qui indique que l'on recherche tous les mots dont le Part-of-Speech (POS) est de type "nom commun" (NOUN) : 

In [5]:
# un premier pattern très simple
pattern1 = [[{"POS": "NOUN"}]]

Regardons ce que spaCy trouve comme matches (noms) dans notre texte : 

In [6]:
def match_pattern(pattern):
    """Take a pattern as input and return matches (with longest match)"""
    matcher = Matcher(nlp.vocab)
    matcher.add("", pattern)
    matches = matcher(doc)
    spans = [doc[start:end] for _, start, end in matches]
    for span in spacy.util.filter_spans(spans):
      print(span)

match_pattern(pattern1)

trained
prototype
intelligence
dialogue
ChatGPT
modèle
langage
continu
grâce
utilisation
techniques
apprentissage
apprentissage
renforcement
réponses
questions
phrases
textes
articles
conversations
humains
textes
ensemble
contraintes
ton
style
sujet
applications
génération
sous-titres
vidéos
création
chatbots
raison
capacités
prototype
inquiétudes
craints
détournements
fins
malveillantes
risques
plagiat
monde
emplois
secteurs
préoccupations
matière
sécurité
confidentialité
modèle
textes
informations
novembre
version
large
exposition
accueil
exactitude
janvier
millions
comptes


spaCy trouve une soixantaine de noms, dont quelques faux positifs.

Essayons à présent un pattern légèrement plus complexe : 

In [7]:
pattern2 = [[
            {"POS": "DET"}, 
            {"POS": "NOUN"},
            {"POS": "ADJ"}
           ]]

match_pattern(pattern2)

l'intelligence artificielle
ses capacités multiples
le monde académique
des informations trompeuses
une version gratuite
son exactitude factuelle


Il est possible de rendre certains éléments optionnels ou de les répéter avec les opérateurs ? (0 ou 1), * (0 ou plus) et + (1 ou plus) ainsi que {1} (exactement 1) etc.

In [8]:
pattern3 = [[
            {"POS": "DET", "OP": "?"}, 
            {"POS": "NOUN", "OP": "{1}"},
            {"POS": "ADJ", "OP": "+"}
           ]]

match_pattern(pattern3)

l'intelligence artificielle
apprentissage supervisé
ses capacités multiples
le monde académique
des informations trompeuses
une version gratuite
exposition médiatique
son exactitude factuelle
comptes enregistrés


Il est également possible de fournir une liste de patterns : 

In [9]:
patterns = [
    [{"POS": "NOUN"}, {"POS": "ADJ"}],
    [{"POS": "ADJ"}, {"POS": "NOUN"}],
    [{"POS": "PROPN"}]
]

match_pattern(patterns)

Chat
Pre
-
Transformer
intelligence artificielle
OpenAI
GPT
OpenAI
apprentissage supervisé
diverses applications
capacités multiples
monde académique
faux textes
informations trompeuses
Lancé
version gratuite
Internet
ChatGPT
exposition médiatique
exactitude factuelle
comptes enregistrés


Ceci permet d'inclure les adjectifs placés avant le nom, ainsi que les noms propres.

## Travail à remettre

Choisissez un texte en français de longueur moyenne (entre 500 et 1000 mots) sur un sujet au choix, par exemple issu de Wikipédia, et remplacez le texte d'exemple ci-dessus. Vous pouvez travailler directement dans la copie du notebook enregistrée sur votre Google Drive, ou bien dans un fichier local. 

Enrichissez la liste de patterns pour couvrir un maximum de syntagmes nominaux valides, tout en excluant les autres constructions syntaxiques. Il faut donc trouver un équilibre en bruit et silence pour minimiser à la fois les faux négatifs (syntagmes valides mais non reconnus par le matcher) et les faux positifs (matches reconnus par erreur qui viennent "polluer" la liste des résultats).

Appliquez ensuite deux des autres analyses de spaCy vues lors du TP (tokenisation, lemmatization, POS-tagging, parsing, named-entity recognition...) à votre texte et évaluez les résultats obtenus. 

Exécutez ces premières analyses avec le modèle `fr_core_news_sm`. Est-il à la hauteur pour les tâches choisies ? Expérimentez ensuite avec (au moins) un modèle plus conséquent : `fr_core_news_md` (medium), `fr_core_news_lg` (large) ou `fr_dep_news_trf` (transformer) pour voir si cela améliore les résultats. N'hésitez pas à vous référer à la [documentation](https://spacy.io/models/fr) pour plus de détails.

Une fois que vous êtes satisfaits des résultats obtenus, rédigez un court rapport (5 pages maximum) en PDF pour expliquer le contexte du TP, le texte choisi, votre démarche ainsi que les analyses effectuées et les performances des différents modèles utilisés pour le français.

Le rapport est à remettre sur l'UV pour le **lundi 26 mai 23h59** au plus tard. Il s'agit d'un travail **strictement individuel** qui ne peut donc PAS se faire en groupe.